# Imports data from folder on a local storage 
# Ouputs the probability of error data when the threshold is estimated by three different approaches.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import math
import matplotlib.pyplot as plt
from scipy import signal
import statistics as stats
import os
from scipy.stats import norm
from numpy.random import normal
from sklearn.mixture import GaussianMixture as GMM
#from sklearn.mixture import BayesianGaussianMixture as BGMM

In [2]:
# Design Variables
N = 6000    # OFDM samples per pseudonym bit (p-bit)
K = 32      # p-bits in a Pseudonym message or packet

In [3]:
# Load collected IQ samples from file
def readCom(file_path):
    return np.fromfile(file_path, dtype=np.complex64)

In [4]:
# Extract folders
def Extract_Folders(x):
    r = []
    for root, dirs, files in os.walk(x):
        r.append(dirs)
    return r

In [5]:
## Compute the average p-bit power for p-bits in the received pseudonym.
## Use Expectation Maximization(EM) in Gaussian Mixture Model for parameter estimation.
## Estimate the mean and variance of the distributions for p-bit '0' and '1' using Expectation Maximization(EM) 
#           in one-dimensional Gaussian Mixture Model with two parameters.
## Determine the threshold using the Bayesian Theorem

def EM_Threshold(x,y):
    data = []

    # Compute the average power for each p-bit in the pseudonym and store them in the data variable
    for j in range(len(x)//N):
        data.append((10**6)*np.mean(abs(x[j*N:(j+1)*N])**2))
        data.append((10**6)*np.mean(abs(y[j*N:(j+1)*N])**2))
    
    # Use the Gaussian Mixture model in sklearn to estimate the mean and variances of the two distributions.
    sort_data = np.array(data).reshape(-1, 1)
    gmm = GMM(n_components = 2, max_iter=1000000, random_state=10, covariance_type = 'spherical', tol = 1e-6, weights_init = [0.5,0.5])
    mean = gmm.fit(sort_data).means_  
    covs  = gmm.fit(sort_data).covariances_
    if mean[1][0] < mean[0][0]:
        mu_0 = mean[1][0]
        mu_1 = mean[0][0]
        var_0 = covs[1]
        var_1 = covs[0]
    else:
        mu_0 = mean[0][0]
        mu_1 = mean[1][0]
        var_0 = covs[0]
        var_1 = covs[1]
    
    # Use Bayesian Decision rule to estimate the threshold.
    a = var_1 - var_0
    b = 2*(mu_1*var_0 - mu_0*var_1)
    r = var_0*var_1*np.log(var_1/var_0)
    c = var_1*mu_0**2 - var_0*mu_1**2 -r
    threshold = (-b+np.sqrt(b**2 - 4*a*c))/(2*a)
    return threshold/(10**6)

In [6]:
# Heuristic Approach to threshold estimation
# Calculate the average p-bit power in for all p-bits in the pseudonym
# Remove 10 average p-bit powers from below and from above so that we can minimize the effect of "outliers".
# Returns the estimated heuristic threshold

def Heuristic_Threshold(x,y):
    sort_data = []
    for i in range(len(x)//N):
        packet_powerx = np.mean(abs(x[i*N:(i+1)*N])**2)
        sort_data.append(packet_powerx)
        packet_powery = np.mean(abs(y[i*N:(i+1)*N])**2)
        sort_data.append(packet_powery)
        z_data = sorted(sort_data, reverse = True)
    return stats.mean(z_data[11:55])

In [7]:
## Compute the average p-bit power for all p-bits received at each Eb/N0
## Using the known transmitted pseudonym sequence, group these average p-bit powers into p-bit '0' and '1'.
## Calculate the statisitical mean and variance of the two groups 
## Calculate the threshold using the Bayes Theorem

def Ideal_Threshold(x,y):
    data_0 = []
    data_1 = []

    for i in range(len(x)//N):
        data_0.append((10**6)*np.mean(abs(x[i*N:(i+1)*N])**2))
        data_1.append((10**6)*np.mean(abs(y[i*N:(i+1)*N])**2))

    mu_0 = stats.mean(data_0)
    mu_1 = stats.mean(data_1)
    var_0 = np.var(data_0)
    var_1 = np.var(data_1)
    
    a = var_1 - var_0
    b = 2*(mu_1*var_0 - mu_0*var_1)
    r = var_0*var_1*np.log(var_1/var_0)
    c = var_1*mu_0**2 - var_0*mu_1**2 -r

    threshold = (-b+np.sqrt(b**2 - 4*a*c))/(2*a)
    return threshold/(10**6)

In [8]:
## PURPOSE: calculates the average probability of Pseudonym bit error by comparing the average power of 
##          N data bits with the threshold value. 
## We compare the three estimated thresholds
def Calculate_BER(x,y):
    ideal_threshold = Ideal_Threshold(x,y)
    Pseud_BER_EM = 0
    Pseud_BER_Heuristic = 0
    Pseud_BER_Ideal = 0
    for i in range(len(x)//(N*K)):
        x_truncated = x[i*N*K:(i+1)*N*K]
        y_truncated = y[i*N*K:(i+1)*N*K]
        num_errors_em = 0
        num_errors_heuristic = 0
        num_errors_ideal = 0
        em_threshold = EM_Threshold(x_truncated,y_truncated)
        heuristic_threshold = Heuristic_Threshold(x_truncated,y_truncated)

        for j in range(len(x_truncated)//N):
            Av_power_0 = np.mean(abs(x_truncated[j*N:(j+1)*N])**2)
            Av_power_1 = np.mean(abs(y_truncated[j*N:(j+1)*N])**2)
            if Av_power_0 > ideal_threshold:
                num_errors_ideal += 1 
            if Av_power_1 < ideal_threshold:
                num_errors_ideal += 1
                
            if Av_power_0 > em_threshold:
                num_errors_em += 1 
            if Av_power_1 < em_threshold:
                num_errors_em += 1
                
            if Av_power_0 > heuristic_threshold:
                num_errors_heuristic += 1 
            if Av_power_1 < heuristic_threshold:
                num_errors_heuristic += 1 
                
        Pseud_BER_Ideal += num_errors_ideal/(2*K)
        Pseud_BER_EM += num_errors_em/(2*K)
        Pseud_BER_Heuristic += num_errors_heuristic/(2*K)

    return Pseud_BER_Ideal/(len(x)//(N*K)), Pseud_BER_EM/(len(x)//(N*K)), Pseud_BER_Heuristic/(len(x)//(N*K))

In [9]:
# PURPOSE: Loop in each folder corresponding to Eb/N0 and 
#          calculate the average probability of Pseudonym bit error
def Calculate_PBER_ModIndex(x):
    folders = Extract_Folders(x)[0]
    Pseudonym_BER_Ideal = []
    Pseudonym_BER_EM = []
    Pseudonym_BER_Heuristic = []
    for i in range(len(folders)):
        fold = folders[i]
        
        ## read data file corresponding to each Eb/N0 values
        pseudonym_0 = readCom(""+str(x)+"/"+str(folders[i])+"/pseud0")
        pseudonym_1 = readCom(""+str(x)+"/"+str(folders[i])+"/pseud1")

        ## calculate probability of pseudonym bit error
        ideal, em, heuristic = Calculate_BER(pseudonym_0,pseudonym_1)
        Pseudonym_BER_Ideal.append(round(ideal,4))
        Pseudonym_BER_EM.append(round(em,4))
        Pseudonym_BER_Heuristic.append(round(heuristic,4))
        
    # sort values
    BER_ideal = sorted(Pseudonym_BER_Ideal, reverse = True)
    BER_em = sorted(Pseudonym_BER_EM, reverse = True)
    BER_heuristic = sorted(Pseudonym_BER_Heuristic, reverse = True)
    
    # print values
    print("P-bit error for Ideal:",np.transpose(np.array(BER_ideal)))
    print("P-bit error for EM:",np.transpose(np.array(BER_em)))
    print("P-bit error for Heuristic:",np.transpose(np.array(BER_heuristic)))
    
#     # plot the BER for the threshold methods
#     plt.plot(BER_ideal)
#     plt.plot(BER_em)
#     plt.plot(BER_heuristic)
#     plt.legend(("Ideal", "EM", "Heuristic"), loc = "upper right")
#     plt.show()
   
    return BER_ideal,BER_em,BER_heuristic

In [ ]:
# Place the folder name containting the experimental data
Calculate_PBER_ModIndex('Phantom_data_10')